In [1]:
# encoding: utf-8
import math
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import cross_validation  
from sklearn.neighbors import KNeighborsClassifier
 
"""
機器學習做鐵達尼號分析
"""

'\n\xe6\xa9\x9f\xe5\x99\xa8\xe5\xad\xb8\xe7\xbf\x92\xe5\x81\x9a\xe9\x90\xb5\xe9\x81\x94\xe5\xb0\xbc\xe8\x99\x9f\xe5\x88\x86\xe6\x9e\x90\n'

In [2]:
train_data_path = '/Users/macbook/Desktop/code/Simple-Machine-Learning/train.csv'
train_data = pd.read_csv(train_data_path,header=0)

n_train_data = train_data.iloc[:,[0,4]]   #挑想要哪幾欄

#女性存活率
female_tourist = len(train_data[train_data['Sex'] == 'female'])  
female_survived = len(train_data[(train_data['Sex'] == 'female') & (train_data['Survived'] == 1)])  
print female_survived * 1.0 / female_tourist  


0.742038216561


In [3]:
#新增一欄Gender把Sex中女性為1,男性為0
#train_data['Gender'] = train_data['Sex'].map({'female': 1, 'male': 0}).astype(int) 
#astype把它變成int
train_data['Sex'] = train_data['Sex'].map({'female': 1, 'male': 0}).astype(int) 

In [4]:
#把不必要的欄位刪除(Ticket,Name,Cabin,Embarked) 

train_data = train_data.drop(['Ticket','Name','Cabin','Embarked'],axis=1) #axis=0:代表橫軸 axis=1:代表縱軸

#分辨該欄有沒有值，有值為false,無值為True
np.isnan(train_data['Age'])

0      False
1      False
2      False
3      False
4      False
5       True
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17      True
18     False
19      True
20     False
21     False
22     False
23     False
24     False
25     False
26      True
27     False
28      True
29      True
       ...  
861    False
862    False
863     True
864    False
865    False
866    False
867    False
868     True
869    False
870    False
871    False
872    False
873    False
874    False
875    False
876    False
877    False
878     True
879    False
880    False
881    False
882    False
883    False
884    False
885    False
886    False
887    False
888     True
889    False
890    False
Name: Age, dtype: bool

In [5]:
#把Age中的所有數值放入array
m = np.ma.masked_array(train_data['Age'], np.isnan(train_data['Age']))
m

masked_array(data = [22.0 38.0 26.0 35.0 35.0 -- 54.0 2.0 27.0 14.0 4.0 58.0 20.0 39.0 14.0
 55.0 2.0 -- 31.0 -- 35.0 34.0 15.0 28.0 8.0 38.0 -- 19.0 -- -- 40.0 -- --
 66.0 28.0 42.0 -- 21.0 18.0 14.0 40.0 27.0 -- 3.0 19.0 -- -- -- -- 18.0
 7.0 21.0 49.0 29.0 65.0 -- 21.0 28.5 5.0 11.0 22.0 38.0 45.0 4.0 -- --
 29.0 19.0 17.0 26.0 32.0 16.0 21.0 26.0 32.0 25.0 -- -- 0.83 30.0 22.0
 29.0 -- 28.0 17.0 33.0 16.0 -- 23.0 24.0 29.0 20.0 46.0 26.0 59.0 -- 71.0
 23.0 34.0 34.0 28.0 -- 21.0 33.0 37.0 28.0 21.0 -- 38.0 -- 47.0 14.5 22.0
 20.0 17.0 21.0 70.5 29.0 24.0 2.0 21.0 -- 32.5 32.5 54.0 12.0 -- 24.0 --
 45.0 33.0 20.0 47.0 29.0 25.0 23.0 19.0 37.0 16.0 24.0 -- 22.0 24.0 19.0
 18.0 19.0 27.0 9.0 36.5 42.0 51.0 22.0 55.5 40.5 -- 51.0 16.0 30.0 -- --
 44.0 40.0 26.0 17.0 1.0 9.0 -- 45.0 -- 28.0 61.0 4.0 1.0 21.0 56.0 18.0 --
 50.0 30.0 36.0 -- -- 9.0 1.0 4.0 -- -- 45.0 40.0 36.0 32.0 19.0 19.0 3.0
 44.0 58.0 -- 42.0 -- 24.0 28.0 -- 34.0 45.5 18.0 2.0 32.0 26.0 16.0 40.0
 24.0 35.0 22.0 30.0

In [6]:
#用numpy把平均數算出來
mean = np.mean(m).astype(int)
mean

29

In [7]:
"""
    意味著
    for x in train_data['Age']:
        if np.isnan(train_data['Age']) == true: x = mean;
        #else: x = x
"""
train_data['Age'] = train_data['Age'].map(lambda x : mean if np.isnan(x) else x) 
train_data['Fare'] = train_data['Fare'].map(lambda x : 0 if np.isnan(x) else int(x)).astype(int) 

In [8]:
#把每位乘客所有的值array起來
x = train_data.values  
x

array([[   1.,    0.,    3., ...,    1.,    0.,    7.],
       [   2.,    1.,    1., ...,    1.,    0.,   71.],
       [   3.,    1.,    3., ...,    0.,    0.,    7.],
       ..., 
       [ 889.,    0.,    3., ...,    1.,    2.,   23.],
       [ 890.,    1.,    1., ...,    0.,    0.,   30.],
       [ 891.,    0.,    3., ...,    0.,    0.,    7.]])

In [9]:
#把Survived的值array起來
y = train_data['Survived'].values
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1,

In [10]:
#把survived那欄拿掉: np.delete(x,第幾欄,axis=1) axis=0:代表橫軸 axis=1:代表縱軸
x = np.delete(x,1,axis=1) 
x

array([[   1.,    3.,    0., ...,    1.,    0.,    7.],
       [   2.,    1.,    1., ...,    1.,    0.,   71.],
       [   3.,    3.,    1., ...,    0.,    0.,    7.],
       ..., 
       [ 889.,    3.,    1., ...,    1.,    2.,   23.],
       [ 890.,    1.,    0., ...,    0.,    0.,   30.],
       [ 891.,    3.,    0., ...,    0.,    0.,    7.]])

In [11]:
#把train_data中一部分拿出來做訓練，再把另一部分拿來測試，再之後要針對每個分類器做出model
#x_train 為已知train_data的feature值, x_test為假設未知的feature
#y_train 為已知train_data的lebal值, y_test為假設未知的lebal
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x,y,test_size=0.3,random_state=1)  

In [12]:
x_train

array([[ 115.,    3.,    1., ...,    0.,    0.,   14.],
       [ 875.,    2.,    1., ...,    1.,    0.,   24.],
       [  77.,    3.,    0., ...,    0.,    0.,    7.],
       ..., 
       [  73.,    2.,    0., ...,    0.,    0.,   73.],
       [ 236.,    3.,    1., ...,    0.,    0.,    7.],
       [  38.,    3.,    0., ...,    0.,    0.,    8.]])

In [13]:
x_test

array([[ 863.,    1.,    1., ...,    0.,    0.,   25.],
       [ 224.,    3.,    0., ...,    0.,    0.,    7.],
       [  85.,    2.,    1., ...,    0.,    0.,   10.],
       ..., 
       [ 240.,    2.,    0., ...,    0.,    0.,   12.],
       [ 485.,    1.,    0., ...,    1.,    0.,   91.],
       [  93.,    1.,    0., ...,    1.,    0.,   61.]])

In [14]:
#使用decision_tree做訓練model
"""
   運用sklearn的library
"""
decision_tree = tree.DecisionTreeClassifier(max_depth=3)  

In [15]:
#下面是Decision Tree分類器的可改變參數
decision_tree.fit(x_train, y_train) 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')

In [16]:
#train model準確度
decision_tree.score(x_test,y_test)

0.78731343283582089

In [17]:
#使用KNN來做訓練model
"""
運用sklearn的library
"""
knn = KNeighborsClassifier(n_neighbors=29)
knn.fit(x_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=29, p=2, weights='uniform')

In [18]:
#下面是KNN分類器的可改變參數
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=29, p=2, weights='uniform')

In [19]:
#train model準確度
knn.score(x_test,y_test)

0.64179104477611937

In [20]:
#model既然已經訓練好了，下一步就是把test_data放進來
#叫出test_data
test_data_path = '/Users/macbook/Desktop/code/Simple-Machine-Learning/test.csv'
test_data = pd.read_csv(test_data_path,header=0)
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [21]:
#把資料弄好
test_data['Sex'] = test_data['Sex'].map({'female': 1, 'male': 0}).astype(int)  
test_data['Fare'] = test_data['Fare'].map(lambda x : 0 if np.isnan(x) else int(x)).astype(int)  

In [22]:
#把不要的欄位去除
test_data = test_data.drop(['Ticket','Name','Cabin','Embarked'],axis=1)  

In [23]:
#Age算平均數，把空白資料補起來
m = np.ma.masked_array(test_data['Age'], np.isnan(test_data['Age']))  
mean = np.mean(m).astype(int)  
test_data['Age'] = test_data['Age'].map(lambda x : mean if np.isnan(x) else int(x)) 
mean

30

In [24]:
#開始預測，預測結果會呈現為一個array
predicts = decision_tree.predict(test_data)
predicts

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [25]:
#把id也弄成一個array(比賽要求格式)
ids = test_data['PassengerId'].values   
ids

array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10

In [26]:
"""
    兩種方法
    1. 用pd.DataFrame方式，但會產生index，需事後修改
    2. 用傳統csv寫入資料
"""
#第1種方法:pd.DataFrame
result = {'PassengerId': ids,
        'Survived': predicts,}
result_df = pd.DataFrame(result)
result_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [27]:
#pandas寫入檔案方法
result_path = '/Users/macbook/Desktop/code/Simple-Machine-Learning/result_df.csv'
result_df.to_csv(result_path)

In [28]:
#第二種方法:import csv
import csv

In [29]:
#開啟一個空白檔案，給予寫入權限（wb）
#給予暫時的寫入空間
result_csv = open('result_csv.csv','wb')
w_result_csv = csv.writer(result_csv)  

In [30]:
#writerow寫入一行的columns名字
w_result_csv.writerow(["PassengerId","Survived"])  

In [31]:
#zip函式功用
zip(ids, predicts)

[(892, 0),
 (893, 1),
 (894, 0),
 (895, 0),
 (896, 1),
 (897, 0),
 (898, 1),
 (899, 0),
 (900, 1),
 (901, 0),
 (902, 0),
 (903, 0),
 (904, 1),
 (905, 0),
 (906, 1),
 (907, 1),
 (908, 0),
 (909, 0),
 (910, 1),
 (911, 1),
 (912, 0),
 (913, 1),
 (914, 1),
 (915, 0),
 (916, 1),
 (917, 0),
 (918, 1),
 (919, 0),
 (920, 0),
 (921, 0),
 (922, 0),
 (923, 0),
 (924, 1),
 (925, 1),
 (926, 0),
 (927, 0),
 (928, 1),
 (929, 1),
 (930, 0),
 (931, 0),
 (932, 0),
 (933, 0),
 (934, 0),
 (935, 1),
 (936, 1),
 (937, 0),
 (938, 0),
 (939, 0),
 (940, 1),
 (941, 1),
 (942, 0),
 (943, 0),
 (944, 1),
 (945, 1),
 (946, 0),
 (947, 0),
 (948, 0),
 (949, 0),
 (950, 0),
 (951, 1),
 (952, 0),
 (953, 0),
 (954, 0),
 (955, 1),
 (956, 0),
 (957, 1),
 (958, 1),
 (959, 0),
 (960, 0),
 (961, 1),
 (962, 1),
 (963, 0),
 (964, 1),
 (965, 0),
 (966, 1),
 (967, 0),
 (968, 0),
 (969, 1),
 (970, 0),
 (971, 1),
 (972, 1),
 (973, 0),
 (974, 0),
 (975, 0),
 (976, 0),
 (977, 0),
 (978, 1),
 (979, 1),
 (980, 1),
 (981, 1),
 (982, 1),

In [32]:
#接著把所有資料寫入，關掉檔案
w_result_csv.writerows(zip(ids, predicts))  
result_csv.close()   